In [7]:
from fastapi import FastAPI
from sqlmodel import Session, select, SQLModel
from sqlalchemy.exc import OperationalError
from models import Epic, TimeLog, User, engine, create_db
import datetime

In [8]:
app = FastAPI()
session = Session(engine)

In [9]:
def string_to_datetime(date_string):
    date = datetime.datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S.%fZ")
    return date

In [10]:
def datetime_to_string(date_date):
    date_string = date_date.strftime("%m/%d/%Y, %H:%M:%S")
    return date_string

In [11]:
def get_user_worktime_by_epic(user_id, epic_name, start_time, end_time):
    start_time_dt = string_to_datetime(start_time)
    end_time_dt = string_to_datetime(end_time)
    statement = select(TimeLog)\
        .where(TimeLog.user_id == user_id)\
        .where(TimeLog.epic_name == epic_name)\
        .where(string_to_datetime(TimeLog.start_time) >= start_time_dt)\
        .where(string_to_datetime(TimeLog.end_time) <= end_time_dt)
    results = session.exec(statement).all()
    by_user = []
    for result in results:
        starting_time = string_to_datetime(result.start_time)
        ending_time = string_to_datetime(result.end_time)
        working_time = ending_time - starting_time
        by_user.append(working_time)
    l_sum = sum(by_user, datetime.timedelta())
    work_time_sum = str(l_sum)
    msg = f"""total work time spent by user {user_id} from {start_time} to {end_time} on epic {epic_name} is {work_time_sum}
            """    
    return msg